## Support Vector Machine Models

## Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


In [2]:
images_train = np.load("../data/images/images_train.npy")
images_test = np.load("../data/images/images_test.npy")
num_samples_train = images_train.shape[0]
num_samples_test = images_test.shape[0]

images_train = images_train.reshape(num_samples_train, -1)
images_test = images_test.reshape(num_samples_test, -1)

hog_train = np.load("../data/hog/hog_train.npy").reshape(num_samples_train, -1)
hog_train = hog_train * 255
hog_test = np.load("../data/hog/hog_test.npy").reshape(num_samples_test, -1)
hog_test = hog_test * 255

canny_edges_train  = np.load("../data/canny_edges/canny_edges_train.npy").reshape(num_samples_train, -1)
canny_edges_test  = np.load("../data/canny_edges/canny_edges_test.npy").reshape(num_samples_test, -1)

contours_train = np.load("../data/contours/contours_train.npy").reshape(num_samples_train, -1)
contours_test = np.load("../data/contours/contours_test.npy").reshape(num_samples_test, -1)

y_train = np.load("../data/images/label_ids_train.npy")
y_test = np.load("../data/images/label_ids_test.npy")


In [3]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{2: 'greenstick',
 3: 'comminuted',
 5: 'pathological',
 0: 'hairline',
 8: 'impacted',
 1: 'spiral',
 7: 'oblique',
 4: 'dislocation',
 6: 'longitudinal',
 9: 'avulsion'}

## Model 1: Images Only

In [4]:
model = SVC(max_iter=10000, kernel='linear')
model.fit(images_train, y_train)


SVC(kernel='linear', max_iter=10000)

In [5]:
y_pred = model.predict(images_test)


In [6]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


Accuracy: 0.30714285714285716
Classification Report:


,precision,recall,f1-score,support
hairline,0.083333,0.100000,0.090909,10.000000
spiral,0.142857,0.083333,0.105263,12.000000
greenstick,0.272727,0.562500,0.367347,16.000000
comminuted,0.428571,0.642857,0.514286,14.000000
dislocation,0.555556,0.526316,0.540541,19.000000
pathological,0.250000,0.166667,0.200000,18.000000
longitudinal,0.444444,0.333333,0.380952,12.000000
oblique,0.250000,0.187500,0.214286,16.000000
impacted,0.000000,0.000000,0.000000,9.000000
avulsion,0.250000,0.214286,0.230769,14.000000


## Model 2: HOG Feature Only

In [7]:
model = SVC(max_iter=10000, kernel='linear')
model.fit(hog_train, y_train)


SVC(kernel='linear', max_iter=10000)

In [8]:
y_pred = model.predict(hog_test)


In [9]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


Accuracy: 0.32857142857142857
Classification Report:


,precision,recall,f1-score,support
hairline,0.066667,0.100000,0.080000,10.000000
spiral,0.285714,0.166667,0.210526,12.000000
greenstick,0.529412,0.562500,0.545455,16.000000
comminuted,0.312500,0.714286,0.434783,14.000000
dislocation,0.272727,0.315789,0.292683,19.000000
pathological,0.250000,0.222222,0.235294,18.000000
longitudinal,0.750000,0.250000,0.375000,12.000000
oblique,0.384615,0.312500,0.344828,16.000000
impacted,0.000000,0.000000,0.000000,9.000000
avulsion,0.500000,0.428571,0.461538,14.000000


## Model 3: Canny Edges Feature Only

In [10]:
model = SVC(max_iter=10000, kernel='linear')
model.fit(canny_edges_train, y_train)


SVC(kernel='linear', max_iter=10000)

In [11]:
y_pred = model.predict(canny_edges_test)


In [12]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


Accuracy: 0.2
Classification Report:


,precision,recall,f1-score,support
hairline,0.000000,0.000000,0.000000,10.0
spiral,0.500000,0.083333,0.142857,12.0
greenstick,0.444444,0.250000,0.320000,16.0
comminuted,0.139535,0.428571,0.210526,14.0
dislocation,0.300000,0.473684,0.367347,19.0
pathological,0.166667,0.166667,0.166667,18.0
longitudinal,0.200000,0.083333,0.117647,12.0
oblique,0.500000,0.125000,0.200000,16.0
impacted,0.076923,0.111111,0.090909,9.0
avulsion,0.111111,0.071429,0.086957,14.0


## Model 4: Contours Feature Only

In [13]:
model = SVC(max_iter=10000, kernel='linear')
model.fit(contours_train, y_train)


SVC(kernel='linear', max_iter=10000)

In [14]:
y_pred = model.predict(contours_test)


In [15]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


Accuracy: 0.2
Classification Report:


,precision,recall,f1-score,support
hairline,0.000000,0.000000,0.000000,10.0
spiral,0.222222,0.166667,0.190476,12.0
greenstick,0.090909,0.125000,0.105263,16.0
comminuted,0.205882,0.500000,0.291667,14.0
dislocation,0.205128,0.421053,0.275862,19.0
pathological,0.333333,0.055556,0.095238,18.0
longitudinal,0.500000,0.166667,0.250000,12.0
oblique,0.285714,0.125000,0.173913,16.0
impacted,0.000000,0.000000,0.000000,9.0
avulsion,0.307692,0.285714,0.296296,14.0


## Model 5: Best Combination

In [17]:
X_train = np.hstack((images_train, hog_train, canny_edges_train, contours_train))
X_test = np.hstack((images_test, hog_test, canny_edges_test, contours_test))

In [18]:
model = SVC(max_iter=10000, kernel='linear')
model.fit(X_train, y_train)


SVC(kernel='linear', max_iter=10000)

In [19]:
y_pred = model.predict(X_test)


In [20]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


Accuracy: 0.30714285714285716
Classification Report:


/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 

,precision,recall,f1-score,support
hairline,0.062500,0.100000,0.076923,10.000000
spiral,0.222222,0.166667,0.190476,12.000000
greenstick,0.275862,0.500000,0.355556,16.000000
comminuted,0.421053,0.571429,0.484848,14.000000
dislocation,0.454545,0.526316,0.487805,19.000000
pathological,0.272727,0.166667,0.206897,18.000000
longitudinal,0.333333,0.250000,0.285714,12.000000
oblique,0.375000,0.187500,0.250000,16.000000
impacted,0.000000,0.000000,0.000000,9.000000
avulsion,0.294118,0.357143,0.322581,14.000000
